In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')

s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

In [ ]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')
ca